In [37]:
import os
import json
import psycopg2
from psycopg2 import sql

In [38]:
# Configurações de conexão
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

# Caminho da pasta base
base_path = "./Data/teste/"

## Load Main File

In [32]:


# Estrutura SQL para inserção na tabela load
insert_load_sql = """
INSERT INTO vehicle_tracking_load (ordem, latitude, longitude, datahora, velocidade, linha, datahoraenvio, datahoraservidor)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql = """
INSERT INTO vehicle_tracking_teste_base (
    ordem,
    latitude,
    longitude,
    datahora_epoch,
    datahora,
    velocidade,
    linha,
    datahoraenvio_epoch,
    datahoraservidor_epoch,
    datahoraservidor,
    datahoraenvio,
    geom
)
SELECT
    ordem,
    latitude::DOUBLE PRECISION,
    longitude::DOUBLE PRECISION,
    datahora::BIGINT / 1000 AS datahora_epoch,
    TO_TIMESTAMP(datahora::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahora,
    velocidade::INT,
    linha,
    datahoraenvio::BIGINT / 1000 AS datahoraenvio_epoch,
    datahoraservidor::BIGINT / 1000 AS datahoraservidor_epoch,
    TO_TIMESTAMP(datahoraservidor::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahoraservidor,
    TO_TIMESTAMP(datahoraenvio::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahoraenvio,
    ST_SetSRID(ST_MakePoint(longitude::DOUBLE PRECISION, latitude::DOUBLE PRECISION), 4326)::geography
FROM vehicle_tracking_load
"""


In [33]:
# Função para truncar a tabela load
def truncate_load_table(cur):
    cur.execute("TRUNCATE TABLE vehicle_tracking_load")
    conn.commit()

# Função para processar cada arquivo JSON
def process_json_file(file_path, cur):
    with open(file_path, 'r') as file:
        data_list = json.load(file)
        print(data_list[:10])
        for data in data_list:
            cur.execute(insert_load_sql, (
                data['ordem'],
                data['latitude'].replace(',', '.'),
                data['longitude'].replace(',', '.'),
                data['datahora'],
                data['velocidade'],
                data['linha'],
                data['datahoraenvio'],
                data['datahoraservidor']
            ))
    conn.commit()

    # Inserir dados na tabela final
    cur.execute(insert_vehicle_tracking_sql)
    conn.commit()

    # Truncar a tabela load para o próximo arquivo
    truncate_load_table(cur)



In [34]:
# Percorrer todas as subpastas e arquivos JSON
processed_files = []

cur = conn.cursor()
for root, dirs, files in os.walk(base_path):
    for file in files:
        print('Processing', file)
        if file.endswith(".json") and file not in processed_files and file.startswith('2024'):
            file_path = os.path.join(root, file)
            processed_files.append(file)
            process_json_file(file_path, cur)
            

# Fechar a conexão
cur.close()
conn.close()

Processing 2024-05-11_06.json
[{'ordem': 'B32598', 'latitude': '-22,90825', 'longitude': '-43,19975', 'datahora': '1715414388000', 'velocidade': '66', 'linha': '342', 'datahoraenvio': '1715414400000', 'datahoraservidor': '1715414404000'}, {'ordem': 'B32731', 'latitude': '-22,81843', 'longitude': '-43,39297', 'datahora': '1715414389000', 'velocidade': '0', 'linha': '386', 'datahoraenvio': '1715414400000', 'datahoraservidor': '1715414404000'}, {'ordem': 'D87860', 'latitude': '-22,85296', 'longitude': '-43,39296', 'datahora': '1715414390000', 'velocidade': '44', 'linha': '2336', 'datahoraenvio': '1715414400000', 'datahoraservidor': '1715414405000'}, {'ordem': 'D17512', 'latitude': '-22,8855', 'longitude': '-43,22856', 'datahora': '1715414391000', 'velocidade': '66', 'linha': '2336', 'datahoraenvio': '1715414400000', 'datahoraservidor': '1715414405000'}, {'ordem': 'D87894', 'latitude': '-22,9017', 'longitude': '-43,55827', 'datahora': '1715414397000', 'velocidade': '0', 'linha': '2336', 'd

## Load treino/teste files

In [31]:

# Função para truncar a tabela load
def truncate_load_table(cur):
    cur.execute("TRUNCATE TABLE vehicle_tracking_treino_latlong_load; TRUNCATE TABLE vehicle_tracking_treino_datahora_load;")
    conn.commit()

insert_load_latlong_sql = """
INSERT INTO vehicle_tracking_treino_latlong_load (id, ordem, latitude, longitude, linha, filename)
VALUES (%s, %s, %s, %s, %s, %s)
"""

insert_load_datahora_sql = """
INSERT INTO vehicle_tracking_treino_datahora_load (id, ordem, datahora, linha, filename)
VALUES (%s, %s, %s, %s, %s)
"""


# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql_treino_latlog = """
INSERT INTO vehicle_tracking_teste_latlong (
    id,
    ordem,
    latitude,
    longitude,
    linha,
    geom,
    filename
)
SELECT
    id,
    ordem,
    latitude::DOUBLE PRECISION,
    longitude::DOUBLE PRECISION,
    linha,
    ST_SetSRID(ST_MakePoint(longitude::DOUBLE PRECISION, latitude::DOUBLE PRECISION), 4326)::geography,
    filename
FROM vehicle_tracking_treino_latlong_load

"""

# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql_treino_datahora = """
INSERT INTO vehicle_tracking_teste_datahora (
    id,
    ordem,
    datahora_epoch,
    datahora,
    linha,
    filename
)
SELECT
    id,
    ordem,
    datahora::bigint,
    TO_TIMESTAMP(datahora::BIGINT / 1000)::TIMESTAMP WITHOUT TIME ZONE AS datahora,
    linha,
    filename
FROM vehicle_tracking_treino_datahora_load
"""



In [32]:
def process_json_file(file_path, filename, cur):
    with open(file_path, 'r') as file:
        data_list = json.load(file)
        print(data_list[:10])
        for data in data_list:
            if 'datahora' in data.keys():
                cur.execute(insert_load_datahora_sql, (data['id'], data['ordem'], data['datahora'], data['linha'], filename))
            else:
                cur.execute(insert_load_latlong_sql, (data['id'], data['ordem'], data['latitude'].replace(',', '.'), data['longitude'].replace(',', '.'), data['linha'], filename))
          # Inserir dados na tabela final

        if 'datahora'in data_list[0].keys():
            cur.execute(insert_vehicle_tracking_sql_treino_datahora)
            conn.commit()
        else:
            cur.execute(insert_vehicle_tracking_sql_treino_latlog)
            conn.commit()

        # Truncar a tabela load para o próximo arquivo
        truncate_load_table(cur)

In [33]:
# Percorrer todas as subpastas e arquivos JSON
processed_files = []

cur = conn.cursor()
for root, dirs, files in os.walk(base_path):
    for file in files:
        print('Processing', file)
        if file.endswith(".json") and file not in processed_files and file.startswith('teste'):
            file_path = os.path.join(root, file)
            processed_files.append(file)
            process_json_file(file_path, file, cur)

    conn.commit()

# Fechar a conexão
cur.close()
conn.close()

Processing 2024-05-16_06.json
Processing 2024-05-16_07.json
Processing 2024-05-16_10.json
Processing 2024-05-16_11.json
Processing 2024-05-16_14.json
Processing 2024-05-16_15.json
Processing 2024-05-16_18.json
Processing 2024-05-16_19.json
Processing teste-2024-05-16_08.json
[{'id': 949472760, 'linha': '422', 'ordem': 'C72081', 'datahora': 1715856027000}, {'id': 15524859950, 'linha': '774', 'ordem': 'B27135', 'datahora': 1715856165000}, {'id': 17439983861, 'linha': '422', 'ordem': 'A72041', 'datahora': 1715856960000}, {'id': 47763099882, 'linha': '634', 'ordem': 'B10063', 'datahora': 1715854612000}, {'id': 51210565044, 'linha': '422', 'ordem': 'A72014', 'datahora': 1715857088000}, {'id': 51884239966, 'linha': '309', 'ordem': 'A41313', 'datahora': 1715855303000}, {'id': 56713372640, 'linha': '606', 'ordem': 'B25578', 'datahora': 1715854023000}, {'id': 57590039329, 'linha': '355', 'ordem': 'B27003', 'datahora': 1715856803000}, {'id': 69787612620, 'linha': '483', 'ordem': 'B31066', 'datah

## Load Resposta files

In [39]:

# Função para truncar a tabela load
def truncate_load_table(cur):
    cur.execute("TRUNCATE TABLE vehicle_tracking_treino_latlong_resposta_load; TRUNCATE TABLE vehicle_tracking_treino_datahora_resposta_load;")
    conn.commit()

insert_load_latlong_sql = """
INSERT INTO vehicle_tracking_treino_latlong_resposta_load (id, latitude, longitude, filename)
VALUES (%s, %s, %s, %s)
"""

insert_load_datahora_sql = """
INSERT INTO vehicle_tracking_treino_datahora_resposta_load (id, datahora, filename)
VALUES (%s, %s, %s)
"""


# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql_treino_latlog = """
INSERT INTO vehicle_tracking_teste_latlong_resposta (
    id,
    latitude,
    longitude,
    filename
)
SELECT
    id,
    latitude::DOUBLE PRECISION,
    longitude::DOUBLE PRECISION,
    filename
FROM vehicle_tracking_treino_latlong_resposta_load

"""

# Query de Insert com Conversão de Tipos
insert_vehicle_tracking_sql_treino_datahora = """
INSERT INTO vehicle_tracking_teste_datahora_resposta (
    id,
    datahora_epoch,
    filename
)
SELECT
    id,
    datahora::bigint,
    filename
FROM vehicle_tracking_treino_datahora_resposta_load
"""



In [40]:
def process_json_file(file_path, filename, cur):
    with open(file_path, 'r') as file:
        data_list = json.load(file)
        print(data_list[:10])
        for data in data_list:
            if 'datahora' in data.keys():
                cur.execute(insert_load_datahora_sql, (data['id'], data['datahora'], filename))
            else:
                cur.execute(insert_load_latlong_sql, (data['id'], data['latitude'].replace(',', '.'), data['longitude'].replace(',', '.'), filename))
          # Inserir dados na tabela final

        if 'datahora'in data_list[0].keys():
            cur.execute(insert_vehicle_tracking_sql_treino_datahora)
            conn.commit()
        else:
            cur.execute(insert_vehicle_tracking_sql_treino_latlog)
            conn.commit()

        # Truncar a tabela load para o próximo arquivo
        truncate_load_table(cur)

In [41]:
# Percorrer todas as subpastas e arquivos JSON
processed_files = []

cur = conn.cursor()
for root, dirs, files in os.walk(base_path):
    for file in files:
        print('Processing', file)
        if file.endswith(".json") and file not in processed_files and file.startswith('resposta'):
            file_path = os.path.join(root, file)
            processed_files.append(file)
            process_json_file(file_path, file,  cur)

    conn.commit()

# Fechar a conexão
cur.close()
conn.close()

Processing 2024-05-16_06.json
Processing 2024-05-16_07.json
Processing 2024-05-16_10.json
Processing 2024-05-16_11.json
Processing 2024-05-16_14.json
Processing 2024-05-16_15.json
Processing 2024-05-16_18.json
Processing 2024-05-16_19.json
Processing teste-2024-05-16_08.json
Processing teste-2024-05-16_12.json
Processing teste-2024-05-16_16.json
Processing teste-2024-05-16_20.json
Processing 2024-05-17_05.json
Processing 2024-05-17_06.json
Processing 2024-05-17_09.json
Processing 2024-05-17_10.json
Processing 2024-05-17_13.json
Processing 2024-05-17_14.json
Processing 2024-05-17_17.json
Processing 2024-05-17_18.json
Processing teste-2024-05-17_07.json
Processing teste-2024-05-17_11.json
Processing teste-2024-05-17_15.json
Processing teste-2024-05-17_19.json
Processing 2024-05-18_06.json
Processing 2024-05-18_07.json
Processing 2024-05-18_10.json
Processing 2024-05-18_11.json
Processing 2024-05-18_14.json
Processing 2024-05-18_15.json
Processing 2024-05-18_18.json
Processing 2024-05-18_